# **Heart Disease Study Notebook**

## Objectives

* Answer business requirement 1:
    * The client is interested to understand how different variables affect Heart Disease presence in patients.

## Inputs

* Generate Dataset: outputs/datasets/collection/Cardiovascular_Disease_Dataset.csv

## Outputs

* Generate code that answers business requirement 1 and can be used to build the Streamlit App 


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [2]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/cd-prediction-pp5/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [3]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'/workspace/cd-prediction-pp5'

# Load Data

In [5]:
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/Cardiovascular_Disease_Dataset.csv")
    )
df.head(5)

,patientid,age,gender,chestpain,restingBP,serumcholestrol,fastingbloodsugar,restingrelectro,maxheartrate,exerciseangia,oldpeak,slope,noofmajorvessels,target
0,103368,53,1,2,171,0,0,1,147,0,5.3,3,3,1
1,119250,40,1,0,94,229,0,1,115,0,3.7,1,1,0
2,119372,49,1,2,133,142,0,0,202,1,5.0,1,0,0
3,132514,43,1,0,138,295,1,1,153,0,3.2,2,2,1
4,146211,31,1,1,199,0,0,2,136,0,5.3,3,2,1


---

# Data Exploration

* We are interested in getting more familiar with the dataset, checking variable type and distribution, missing levels and what these variables mean in a clinical context

In [8]:
from ydata_profiling import ProfileReport
pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()

Render HTML: 100%|██████████| 1/1 [00:00<00:00,  2.53it/s]


From Pandas profiling report we observe:

* Age appears to be unnaturally distributed. This might point to a foult in data records
* There's a lot more male patients than female patients (about 3/1 ratio)
* There's '0' values in total cholesterol count (serumcholestrol). This can't be accurate as low colesterol level is leathal. We will deal with this in Data Cleaning notebook

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
